Collected by passing the ledger files to gpt 4o with the prompt

```
"""
For each transaction in my data where the transaction goes to Assets:Discover:Spent, extract the transaction's information in the following format:

{
    "MCDONALD’S": {"day": "07", "amount": "6.51", "from": "Assets:Discover:Main:Wants:Other"},
    ...
}
"""
```

In [8]:
import sys
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, project_root)

from src.opensearch.client import client, index_name
from src.opensearch.upload import add_index, add_documents

In [12]:
june = {
    "MCDONALD'S": {"day": "07", "amount": "6.51", "from_account": "Assets:Discover:Main:Wants:Other"},
    "MAD MAX TICKETS": {"day": "07", "amount": "28.46", "from_account": "Assets:Discover:Main:Wants:Other"},
    "SQ BERNARDOS BURRITOS": {"day": "08", "amount": "11.22", "from_account": "Assets:Discover:Main:Wants:Other"},
    "SPOTIFY": {"day": "09", "amount": "11.76", "from_account": "Assets:Discover:Main:Wants:Monthly"},
    "WALMART STORE": {"day": "18", "amount": "63.39", "from_account": "Assets:Discover:Main:Needs:Groceries"},
    "CASEYS": {"day": "12", "amount": "38.72", "from_account": "Assets:Discover:Main:Needs:Gas"},
    "PANCHEROS": {"day": "13", "amount": "11.56", "from_account": "Assets:Discover:Main:Wants:Other"},
    "TST DES MOINES BEER": {"day": "14", "amount": "40.26", "from_account": "Assets:Discover:Main:Wants:Other"},
    "QUANTUM FIBER": {"day": "15", "amount": "85.00", "from_account": "Assets:Discover:Main:Needs:Monthly"},
    "TST CHOWRASTHA INDIAN": {"day": "16", "amount": "41.20", "from_account": "Assets:Discover:Main:Wants:Other"},
    "JIMMY JOHNS": {"day": "16", "amount": "22.18", "from_account": "Assets:Discover:Main:Wants:Other"},
    "AMAZON.COM CEREAL FOR DAD": {"day": "17", "amount": "12.12", "from_account": "Assets:Discover:Main:Wants:Other"},
    "HABENEROS MEXICAN": {"day": "21", "amount": "40.00", "from_account": "Assets:Discover:Main:Wants:Other"},
    "AMAZON HBO": {"day": "22", "amount": "10.69", "from_account": "Assets:Discover:Main:Wants:Monthly"}
}

may = {
    "MIYABI 9": {"day": "04", "amount": "70", "from_account": "Assets:Discover:Main:Wants:Other"},
    "DSM PARKING": {"day": "04", "amount": "1.25", "from_account": "Assets:Discover:Main:Wants:Other"},
    "CHATGPT": {"day": "04", "amount": "20", "from_account": "Assets:Discover:Main:Wants:Monthly"},
    "AMAZON PRIME": {"day": "04", "amount": "16.04", "from_account": "Assets:Discover:Main:Needs:Monthly"},
    "QDOBA": {"day": "06", "amount": "22.04", "from_account": "Assets:Discover:Main:Wants:Other"},
    "WALMART": {"day": "07", "amount": "66.96", "from_account": "Assets:Discover:Main:Needs:Groceries"},
    "CULVERS": {"day": "07", "amount": "13.95", "from_account": "Assets:Discover:Main:Wants:Other"},
    "MAD MEATBALL": {"day": "09", "amount": "16", "from_account": "Assets:Discover:Main:Wants:Other"},
    "SPOTIFY": {"day": "09", "amount": "11.76", "from_account": "Assets:Discover:Main:Wants:Monthly"},
    "LUA THC DRINKS": {"day": "10", "amount": "28.87", "from_account": "Assets:Discover:Main:Wants:Other"},
    "JIMMY JOHNS": {"day": "10", "amount": "22.18", "from_account": "Assets:Discover:Main:Wants:Other"},
    "ADIDAS SHOES": {"day": "11", "amount": "115.33", "from_account": "Assets:Discover:FutureWants"},
    "AMAZON EMILY GRAD GIFT": {"day": "12", "amount": "109.50", "from_account": "Assets:Discover:FutureWants"},
    "DOMINOS": {"day": "12", "amount": "17.86", "from_account": "Assets:Discover:Main:Wants:Other"},
    "WALMART GROCERIES": {"day": "14", "amount": "49.02", "from_account": "Assets:Discover:Main:Needs:Groceries"},
    "TARGET EMILY GRAD GIFT FILM": {"day": "14", "amount": "13.90", "from_account": "Assets:Discover:FutureWants"},
    "INTERNET BILL": {"day": "15", "amount": "85", "from_account": "Assets:Discover:Main:Needs:Monthly"},
    "AMAZON EMILY GRAD GIFT FILM": {"day": "15", "amount": "29.92", "from_account": "Assets:Discover:FutureWants"},
    "DESPENSARY": {"day": "17", "amount": "21.40", "from_account": "Assets:Discover:Main:Wants:Other"},
    "PANCHEROS": {"day": "17", "amount": "23.11", "from_account": "Assets:Discover:Main:Wants:Other"},
    "BP DRINKS": {"day": "19", "amount": "6.07", "from_account": "Assets:Discover:Main:Wants:Other"},
    "WALMART GROCERIES": {"day": "21", "amount": "37.86", "from_account": "Assets:Discover:Main:Needs:Groceries"},
    "GAS FROM SAMS": {"day": "21", "amount": "38.04", "from": "Assets:Discover:Main:Needs:Gas"},
    "SAMS GROCERIES": {"day": "21", "amount": "84.57", "from_account": "Assets:Discover:Main:Needs:Groceries"},
    "PALMERS LUNCH": {"day": "21", "amount": "21.06", "from_account": "Assets:Discover:Main:Wants:Other"},
    "AMAZON PRIME HBO MAX": {"day": "22", "amount": "10.69", "from_account": "Assets:Discover:Main:Wants:Monthly"},
    "CHIPOTLE": {"day": "27", "amount": "24.98", "from_account": "Assets:Discover:Main:Wants:Other"},
    "OBSIDIAN": {"day": "27", "amount": "10", "from_account": "Assets:Discover:Main:Wants:Monthly"},
    "BEER GARDEN": {"day": "31", "amount": "39.04", "from_account": "Assets:Discover:Main:Wants:Other"},
    "DOMINOS": {"day": "31", "amount": "26.55", "from_account": "Assets:Discover:Main:Wants:Other"},
    "WALMART ALLERGY MEDS": {"day": "01", "amount": "48.26", "from_account": "Assets:Discover:Main:Needs:Groceries"},
    "GOOGLE GSUITE": {"day": "01", "amount": "12.84", "from_account": "Assets:Discover:Main:Wants:Monthly"},
    "AWS BILL": {"day": "02", "amount": "2.89", "from_account": "Assets:Discover:Main:Wants:Monthly"},
    "INDYCAR TICKETS": {"day": "02", "amount": "308", "from_account": "Assets:Discover:FutureWants"},
    "AMAZON HOUSE UTILITIES": {"day": "02", "amount": "76.01", "from_account": "Assets:Discover:Furniture"},
    "AMAZON PRIME": {"day": "04", "amount": "16.04", "from_account": "Assets:Discover:Main:Needs:Monthly"},
    "CHICK FIL A": {"day": "04", "amount": "11.97", "from_account": "Assets:Discover:Main:Wants:Other"},
    "CHATGPT SUBSCRIPTION": {"day": "04", "amount": "20", "from_account": "Assets:Discover:Main:Wants:Monthly"},
    "WALMART GROCERIES": {"day": "04", "amount": "17.95", "from_account": "Assets:Discover:Main:Needs:Groceries"}
}

add_index(client, index_name)
add_documents(client, index_name, [may, june])

In [11]:
client.indices.delete(index_name)

{'acknowledged': True}

In [166]:
client.indices.get_mapping(index_name)

{'transactions': {'mappings': {'properties': {'amount': {'type': 'float'},
    'day': {'type': 'text'},
    'from': {'type': 'text'},
    'vendor': {'type': 'text'}}}}}